In [1]:
import sys
sys.path.insert(0, '../')

import pdb

import numpy as np
import pandas as pd
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2


from keys.spotify_keys import * 

In [2]:
data = pd.read_csv("../data/raw/raw_dropped_duplicates.zip")

In [3]:
data.shape

(107973, 20)

In [59]:
i = 0
ids = list(data.ID.iloc[i:i + 50])

In [60]:

CLIENT_ID = spotify_keys["CLIENT_ID"]
CLIENT_SECRET = spotify_keys["CLIENT_SECRET"]

credentials = oauth2.SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET)

token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)

res = spotify.tracks(ids)
print(res)

{'tracks': [{'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0b2B3PwcYzQAhuJacmcYgc'}, 'href': 'https://api.spotify.com/v1/artists/0b2B3PwcYzQAhuJacmcYgc', 'id': '0b2B3PwcYzQAhuJacmcYgc', 'name': 'The HU', 'type': 'artist', 'uri': 'spotify:artist:0b2B3PwcYzQAhuJacmcYgc'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'SA', 'SE', 'SG', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'US', 'UY', 'VN', 'ZA'], 'external_urls': {'spotify': 'https://open.spotify.com/album/2ot7qJ6yU06AkwkAn9xeoP'}, 'href': 'https://api.spotify.com/v1/albums/2ot7qJ6yU06AkwkAn9xeoP', 'id': '2ot7qJ

In [56]:
class SpotifyAPIParser:
    
    @classmethod
    def get_track_release_dates(cls, data, start_index, sp):
        # spotify api calls allow for up to 50 queries per call
        # querying tracks requires a list of track IDs
        ids = list(data.ID.iloc[start_index: start_indx + 50])

        res = sp.tracks(ids)
        parsed = cls.parse_api_response(res)
        # parsed = [(ID, album_artists, release_date)]
    
    @classmethod
    def parse_api_response(cls, res):
        track_data = []
        for track in res['tracks']:
            track_data.append(cls.parse_track_data(track))
        track_df = pd.DataFrame(track_data, columns=(
            "ID", 
            "artist_names",
            "artist_ids",
            "release_date",
            "release_date_precision",
        ))
        return track_df
        
    @classmethod        
    def parse_track_data(cls, track):
        ID = track['id']
        artist_names = cls.get_track_artists(track)
        artist_ids = cls.get_track_artist_ids(track)
        release_date = cls.get_track_release_date(track)
        release_date_precision = cls.get_track_release_date_precision(track)
        
        return (
            ID,
            artist_names,
            artist_ids,
            release_date,
            release_date_precision
        )
    
    @classmethod
    def get_track_artists(cls, track):
        try:
            artists = []
            for artist in track['artists']:
                artists.append(artist['name'])
            return ",".join(artists)
        except:
            return np.NaN

    @classmethod
    def get_track_artist_ids(cls, track):
        try:
            artist_ids = []
            for artist in track['artists']:
                artist_ids.append(artist['id'])
            return ",".join(artist_ids)
        except:
            return np.NaN
        
    @classmethod
    def get_track_release_date(cls, track):
        try:
            return track['album']['release_date']
        except:
            return np.NaN
        
    @classmethod
    def get_track_release_date_precision(cls, track):
        try:
            return track['album']['release_date_precision']
        except:
            return np.NaN

In [61]:
track = {'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2egqZI7Cz9JqioUoUBxM9H'},
    'href': 'https://api.spotify.com/v1/artists/2egqZI7Cz9JqioUoUBxM9H',
    'id': '2egqZI7Cz9JqioUoUBxM9H',
    'name': 'Drakum',
    'type': 'artist',
    'uri': 'spotify:artist:2egqZI7Cz9JqioUoUBxM9H'}],
  'available_markets': [],
  'external_urls': {'spotify': 'https://open.spotify.com/album/0OdDdqkPazkYO46ocxqCFx'},
  'href': 'https://api.spotify.com/v1/albums/0OdDdqkPazkYO46ocxqCFx',
  'id': '0OdDdqkPazkYO46ocxqCFx',
  'images': [{'height': 600,
    'url': 'https://i.scdn.co/image/708101f4170c92c558c7028917ab2c569a4c6f60',
    'width': 600},
   {'height': 300,
    'url': 'https://i.scdn.co/image/c8671483b0c0644a9cad1963455b903da12ec4fd',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/bd2cbd012ce86b983921010ccccb01826dfcf3f7',
    'width': 64}],
  'name': 'Around the Oak',
  'release_date': '2011-04-21',
  'release_date_precision': 'day',
  'total_tracks': 3,
  'type': 'album',
  'uri': 'spotify:album:0OdDdqkPazkYO46ocxqCFx'},
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2egqZI7Cz9JqioUoUBxM9H'},
   'href': 'https://api.spotify.com/v1/artists/2egqZI7Cz9JqioUoUBxM9H',
   'id': '2egqZI7Cz9JqioUoUBxM9H',
   'name': 'Drakum',
   'type': 'artist',
   'uri': 'spotify:artist:2egqZI7Cz9JqioUoUBxM9H'}],
 'available_markets': [],
 'disc_number': 1,
 'duration_ms': 238889,
 'explicit': False,
 'external_ids': {'isrc': 'ESAAI0260838'},
 'external_urls': {'spotify': 'https://open.spotify.com/track/1xZAjz98q8zbtASUvkLCMY'},
 'href': 'https://api.spotify.com/v1/tracks/1xZAjz98q8zbtASUvkLCMY',
 'id': '1xZAjz98q8zbtASUvkLCMY',
 'is_local': False,
 'name': 'Around the Oak',
 'popularity': 0,
 'preview_url': None,
 'track_number': 1,
 'type': 'track',
 'uri': 'spotify:track:1xZAjz98q8zbtASUvkLCMY'}

# print(SpotifyAPIParser.parse_track_data(track))
SpotifyAPIParser.parse_api_response(res)


,ID,artist_names,artist_ids,release_date,release_date_precision
0,6J2VvzKwWc2f0JP5RQVZjq,The HU,0b2B3PwcYzQAhuJacmcYgc,2018-10-14,day
1,4HA34COgxgVJ6zK88UN4Ik,Gloryhammer,7dkEByOe0oHqc54qU4hwzV,2019-04-12,day
2,3W6Xik6Xxf06JuUoZSATlD,"Orion's Reign,Minniva","3hRL14TJtJqr7i1IH3D1pU,50KIyycosYOGrBHfLaI1Ju",2018-10-05,day
3,2gGveBaLJQMtJ43X4UL5kH,Brothers of Metal,0WPCPYm5IDNtQjuJOGNPcy,2018-11-09,day
4,1lRF81A1C9QoCgBcEop2zg,Powerwolf,5HFkc3t0HYETL4JeEbDB1v,2018-07-13,day
5,3lUQ6y8XeeaoK2hPydcX9c,Mono Inc.,1189BemPY7h2lgaH1pkzGX,2018-07-27,day
6,6sXsi4DsxtpI4BnfhaG77v,Eluveitie,5X0N2k3qMnI8kSrGJT3kfT,2019-03-15,day
7,3Mz5lXbMBK2tUmKAwAJwi1,Orden Ogan,3t5X2CVDf5mrlIx1SdvWYM,2017-07-07,day
8,7AstJIsEomp0hJyMasOqLN,Battle Beast,7k5jeohQCF20a8foBD9ize,2019-01-18,day
9,1L4UqbKtfM4bk3UdOzrXny,"Timo Tolkki’s Avalon,Todd Michael Hall","6UkYfNV9Y4kp00UT4D2djV,5ZLa4aUwDkoflqCXq81x4A",2019-04-23,day


In [36]:
res['tracks'][0]

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2egqZI7Cz9JqioUoUBxM9H'},
    'href': 'https://api.spotify.com/v1/artists/2egqZI7Cz9JqioUoUBxM9H',
    'id': '2egqZI7Cz9JqioUoUBxM9H',
    'name': 'Drakum',
    'type': 'artist',
    'uri': 'spotify:artist:2egqZI7Cz9JqioUoUBxM9H'}],
  'available_markets': [],
  'external_urls': {'spotify': 'https://open.spotify.com/album/0OdDdqkPazkYO46ocxqCFx'},
  'href': 'https://api.spotify.com/v1/albums/0OdDdqkPazkYO46ocxqCFx',
  'id': '0OdDdqkPazkYO46ocxqCFx',
  'images': [{'height': 600,
    'url': 'https://i.scdn.co/image/708101f4170c92c558c7028917ab2c569a4c6f60',
    'width': 600},
   {'height': 300,
    'url': 'https://i.scdn.co/image/c8671483b0c0644a9cad1963455b903da12ec4fd',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/bd2cbd012ce86b983921010ccccb01826dfcf3f7',
    'width': 64}],
  'name': 'Around the Oak',
  'release_date': '2011-04-21',
  'release_date

In [22]:
for track in res['tracks']:
    print(track["album"]['artists'][0]["name"])
    

Drakum
Northland
Korpiklaani
Eluveitie
In Extremo
Eluveitie
Eluveitie
Eluveitie
Eluveitie
Eluveitie
Týr
Týr
Týr
Týr
Amon Amarth
Amon Amarth
Amon Amarth
Amon Amarth
Amon Amarth
Amon Amarth
Amon Amarth
Northland
Northland
Crimfall
Skyclad
Ensiferum
Ensiferum
Ensiferum
Korpiklaani
Sabaton
Trewa
Skiltron
Týr
Thyrfing
Pertness
Wolfheart
Equilibrium
Tvivel
Trewa
Brymir
Thyrfing
Firkin
Eastern High
Brave the Sea
Wolfheart
Amon Amarth
Heidevolk
Firkin
Aexylium
Aexylium


In [77]:
spotify.categories(country=["US"], limit=50)

{'categories': {'href': 'https://api.spotify.com/v1/browse/categories?country=US&offset=0&limit=50',
  'items': [{'href': 'https://api.spotify.com/v1/browse/categories/toplists',
    'icons': [{'height': 275,
      'url': 'https://t.scdn.co/media/derived/toplists_11160599e6a04ac5d6f2757f5511778f_0_0_275_275.jpg',
      'width': 275}],
    'id': 'toplists',
    'name': 'Top Lists'},
   {'href': 'https://api.spotify.com/v1/browse/categories/summer',
    'icons': [{'height': None,
      'url': 'https://t.scdn.co/images/8e508d7eb5b843a89c368c9507ecc429.jpeg',
      'width': None}],
    'id': 'summer',
    'name': 'Summer'},
   {'href': 'https://api.spotify.com/v1/browse/categories/pop',
    'icons': [{'height': 274,
      'url': 'https://t.scdn.co/media/derived/pop-274x274_447148649685019f5e2a03a39e78ba52_0_0_274_274.jpg',
      'width': 274}],
    'id': 'pop',
    'name': 'Pop'},
   {'href': 'https://api.spotify.com/v1/browse/categories/hiphop',
    'icons': [{'height': 274,
      'url': 